# 硬件
:label:`sec_hardware`

构建具有出色性能的系统需要很好地理解算法和模型，以捕获统计方面的问题。同时，至少对底层硬件有一定的了解也是必不可少的。本节不可替代硬件和系统设计的相关课程。相反，它可以作为理解为什么某些算法比其他算法更高效以及如何实现良好吞吐量的起点。一个好的设计可以很容易地造成性能上数量级的差异。反过来，这也可以在能够训练一个网络（例如，在1周内训练好）和根本无法训练网络（需要3个月来训练）之间产生差异。我们将从研究计算机开始。然后我们将放大以更仔细地查看CPU和GPU。最后，我们将缩小以查看在数据中心或云中多台计算机是如何连接的。

![每个程序员都应该知道的延迟数字。](../img/latencynumbers.png)
:label:`fig_latencynumbers`

不耐烦的读者也许可以通过 :numref:`fig_latencynumbers` 简单了解。它取自科林·斯科特的[互动帖子](https://people.eecs.berkeley.edu/~rcs/research/interactive_latency.html)这很好地概述了过去十年的进展。原来的数字是来自杰夫迪恩的[Stanford talk from 2010](https://static.googleusercontent.com/media/research.google.com/en//people/jeff/Stanford-DL-Nov-2010.pdf)。下面的讨论解释了这些数字的一些基本原理，以及它们如何指导我们设计算法。下面的讨论是非常笼统和粗略的。很明显，它并不能代替一门完整的课程，而只是为了给统计建模者提供足够的信息，让他们做出合适的设计决策。对于计算机体系结构的深入概述，我们建议读者参考 :cite:`Hennessy.Patterson.2011` 或关于该主题的最新课程，例如[Arste Asanovic](http://inst.eecs.berkeley.edu/~cs152/sp19/)。

## 计算机

大多数深度学习研究者和实践者都可以使用一台具有相当数量的内存、计算、某种加速卡（如GPU）的计算机。计算机由以下关键部件组成：

* 一个处理器（也被称为CPU），它能够执行我们给它的程序（除了运行操作系统和许多其他功能之外），通常由8个或更多核心组成。
* 内存（RAM）用于存储和检索计算结果，如权重向量和激活，以及训练数据。
* 一个以太网连接（有时是多个），速度从1 GB/s到100 GB/s不等。在高端服务器上可以找到更高级的互连。
* 将系统连接到一个或多个GPU的高速扩展总线（PCIe）。服务器最多有8个加速卡，通常以高级拓扑连接，而桌面系统则有1个或2个加速卡，具体取决于用户的预算和电源的大小。
* 耐用的存储设备，如磁盘驱动器、固态驱动器，在许多情况下使用PCIe总线连接。它可根据需要高效地将训练数据传输到系统，并根据需要存储中间检查点。

![计算机组件的连接。](../img/mobo-symbol.svg)
:label:`fig_mobo-symbol`

如 :numref:`fig_mobo-symbol` 所示，大多数组件（网络、GPU和存储）通过PCIe总线连接到CPU。它由直接连接到CPU的多个通道组成。例如，AMD的Threadripper 3有64个PCIe 4.0通道，每个通道都能够双向传输16 Gbit/s的数据。内存直接连接到CPU，总带宽高达100gb/s。

当我们在计算机上运行代码时，我们需要将数据转移到处理器（CPU或GPU）执行计算，然后将结果从处理器移回内存和持久存储器。因此，为了获得良好的性能，我们需要确保这一点能够无缝工作，而不会有任何一部分成为主要的瓶颈。例如，如果我们不能足够快地加载图像，处理器将没有任何工作要做。同样地，如果我们不能足够快地将矩阵移动到CPU（或GPU）上，CPU（或GPU）就会无法全力运行。最后，如果我们想在网络上同步多台计算机，网络不应该减慢计算速度。一种选择是交叉通信和计算。让我们更详细地看看各个组件。

## 内存

最基本的内存用于存储需要随时访问的数据。目前，CPU的内存通常为[DDR4](https://en.wikipedia.org/wiki/DDR4_SDRAM)类型，每个模块提供20-25Gb/s的带宽。每个模块都有一条64位宽的总线。通常使用成对的内存模块来允许多个通道。CPU有2到4个内存通道，也就是说，它们的峰值内存带宽在40GB/s到100 GB/s之间。通常每个通道有两个物理BANK。例如AMD的Zen 3T Threadripper有8个插槽。

虽然这些数字令人印象深刻，但实际上，它们只说明了故事的一部分。当我们想要从内存中读取一部分时，我们首先需要告诉内存模块在哪里可以找到信息。也就是说，我们首先需要将*地址*（address）发送到内存。完成后，我们可以选择只读取一条64位记录或一长串记录。后者称为“突发读取”（burst read）。简而言之，向内存发送地址并设置传输大约需要100ns（细节取决于所用内存芯片的特定定时系数），每个后续传输只需要0.2ns。简而言之，第一次读取的成本是后续读取的500倍！请注意，我们每秒最多可以执行10000000次随机读取。这表明我们尽可能避免随机内存访问，而是使用突发读取（和写入）。

当我们考虑到我们有多个物理BANK时，事情就更复杂了。每个BANK都可以独立地读取内存。这意味着两件事。一方面，如果随机读操作均匀分布在内存中，那么有效的随机读操作次数将高达4倍。这也意味着执行随机读取仍然不是一个好主意，因为突发读取的速度也快了4倍。另一方面，由于内存对齐到64位边界，最好将任何数据结构与相同的边界对齐。当设置了适当的标志时，编译器基本上就是[自动化](https://en.wikipedia.org/wiki/Data_structure_alignment)地执行此操作。我们鼓励好奇的读者回顾一个关于DRAM的讲座，比如[Zeshan Chishti](http://web.cecs.pdx.edu/~zeshan/ece585_lec5.pdf)所做的讲座。

GPU内存的带宽要求甚至更高，因为它们的处理单元比CPU多得多。总的来说，解决这些问题有两种选择。首先是使内存总线变得更宽。例如，NVIDIA的RTX 2080 Ti有一条352位宽的总线。这样就可以同时传输更多的信息。其次，GPU使用特定的高性能内存。消费级设备，如NVIDIA的RTX和Titan系列，通常使用[GDDR6](https://en.wikipedia.org/wiki/GDDR6_SDRAM)芯片，总带宽超过500GB/s。另一种选择是使用HBM(高带宽存储器)模块。它们使用截然不同的接口，直接与专用硅片上的GPU连接。这使得它们非常昂贵，通常仅限于高端服务器芯片，如NVIDIA Volta V100系列加速卡。毫不奇怪，GPU内存通常比CPU内存小得多，因为前者的成本更高。就我们的目的而言，它们的性能特征大体上是相似的，只是速度要快得多。就本书而言，我们完全可以忽略细节。它们只在调整GPU核心以获得高吞吐量时才起作用。

## 存储器

我们看到内存的一些关键特性是*带宽*（bandwidth）和*延迟*（latency）。存储设备也是如此，只是差异可能更大。

### 硬盘驱动器

硬盘驱动器（Hard disk drives，HDDs）已经使用了半个多世纪。简单说，它们包含许多旋转的盘片，这些盘片的磁头可以放置在任何给定的磁道上进行读写。高端磁盘在9个盘片上可容纳高达16 TB的容量。硬盘的主要优点之一是相对便宜。它们的许多缺点之一是典型的灾难性故障模式和相对较高的读取延迟。

要理解后者，请考虑一下硬盘驱动器的转速大约为7200 RPM（每分钟转数）。如果它们转速更快，它们就会由于施加在盘子上的离心力而碎裂。在访问磁盘上的特定扇区时，这有一个主要缺点：我们需要等待，直到盘片旋转到位（我们可以移动磁头，但不能加速实际的磁盘）。因此，在请求的数据可用之前可能需要8毫秒。一种常见的表达方式是，硬盘驱动器可以以大约100 IOPs（每秒输入/输出操作）的速度工作。这一数字在过去二十年中基本上没有变化。更糟糕的是，增加带宽同样困难（大约为100-200MB/s）。毕竟，每个磁头读取一个比特磁道，因此比特率只随信息密度的平方根缩放。因此，对于非常大的数据集，HDD正迅速降级为归档存储和低级存储。

### 固态驱动器

固态驱动器（Solid state drives，SSD）使用闪存持久地存储信息。这允许更快地访问存储的记录。现代固态驱动器的IOPs可以达到100000到500000，即比硬盘驱动器快3个数量级。此外，它们的带宽可以达到1-3GB/s，即比硬盘驱动器快一个数量级。这些改进听起来太好了，简直难以置信。实际上，由于固态驱动器的设计方式，它们有以下注意事项:

* 固态驱动器以块（256 KB或更大）存储信息。它们只能作为一个整体来写入，这需要大量的时间。因此，固态驱动器上的按位随机写入性能非常差。同样地，写入数据通常需要大量的时间，因为块必须被读取、擦除，然后用新信息重写。到目前为止，固态驱动器的控制器和固件已经开发出了缓解这种情况的算法。尽管如此，写入速度可能会慢得多，特别是对于QLC（四层单元）固态驱动器。提高性能的关键是维护操作的“队列”。如果可能，优先读取和写入大的块。
* 固态驱动器中的存储单元磨损得比较快（通常在几千次写入之后就已经磨损了）。磨损级别保护算法能够将退化扩展到许多单元。也就是说，不建议将固态驱动器用于交换分区文件或大型日志文件。
* 最后，带宽的大幅增加迫使计算机设计者将固态驱动器直接连接到PCIe总线。能够处理此问题的驱动器称为NVMe（非易失性内存增强），最多可以使用4个PCIe通道。在PCIe 4.0上最高可达8GB/s。

### 云存储

云存储提供了一系列可配置的性能。也就是说，根据用户的选择，虚拟机的存储分配在数量和速度上都是动态的。我们建议用户在延迟太高时（例如，在有许多小记录的训练期间）增加IOPs的配置数。

## CPU

中央处理器（CPU）是任何计算机的核心。它们由许多关键组件组成：能够执行机器代码的*处理器核心*（processor cores）、连接它们的*总线*（bus）（注意，总线会因为处理器型号、各代产品和供应商之间的特定拓扑结构有明显不同）和*缓存*（caches）（允许比从主内存读取更高的带宽和更低的延迟内存访问）。最后，几乎所有的现代CPU都包含*向量处理单元*（vector processing units）来辅助高性能线性代数和卷积运算，因为它们在媒体处理和机器学习中很常见。

![Intel Skylake消费级四核CPU。](../img/skylake.svg)
:label:`fig_skylake`

:numref:`fig_skylake`描述了Intel Skylake消费级四核CPU。它有一个集成的GPU、缓存和一个连接四个核心的环总线。外围设备，如以太网、WiFi、蓝牙、SSD控制器和USB，要么是芯片组的一部分，要么通过PCIe直接连接到CPU。

### 微体系结构

每个处理器核心都由一组相当复杂的组件组成。虽然各代产品和供应商的细节有所不同，但基本功能基本上是标准的。前端加载指令并尝试预测将采用哪条路径（例如，对于控制流）。然后将指令从汇编代码解码为微指令。汇编代码通常不是处理器执行的最低级别代码。相反，复杂指令可以被解码成一组更低级的操作。然后由实际的执行核心处理。后者通常能够同时执行许多操作。例如， :numref:`fig_cortexa77` 的ARM Cortex A77核心可以同时执行多达8个操作。

![ARM Cortex A77 微体系结构](../img/a77.svg)
:label:`fig_cortexa77`

这意味着高效的程序可以在每个时钟周期执行多条指令，前提是它们可以独立执行。并不是所有的单位都是平等的。一些专用于整数指令，而另一些则针对浮点性能进行了优化。为了提高吞吐量，处理器还可以在分支指令中同时遵循多个代码路径，然后丢弃未执行分支的结果。这就是为什么分支预测单元很重要（在前端），只有最有希望的路径才会被跟踪。

### 矢量化

深度学习是非常需要计算机的。因此，为了使CPU适合机器学习，需要在一个时钟周期内执行许多操作。这是通过向量处理单元实现的。它们有不同的名称: 在ARM上叫做NEON, 在x86上被称为[AVX2](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions)。一个常见的功能是它们能够执行单指令多数据（single instruction multiple data，SIMD）操作。 :numref:`fig_neon128` 显示了如何在ARM上的一个时钟周期中完成8个整数加法。

![128位NEON矢量化](../img/neon128.svg)
:label:`fig_neon128`

根据体系结构的选择，此类寄存器最长可达512位，最多可组合64对数字。例如，我们可能会将两个数字相乘，然后将它们与第三个数字相加，这也称为乘加融合（fused multiply-add）。Intel的[OpenVino](https://01.org/openvinotoolkit)使用这些处理器来获得可观的吞吐量，以便在服务器级CPU上进行深度学习。不过请注意，与GPU的能力相比，这个数字完全相形见绌。例如，NVIDIA的RTX 2080Ti拥有4352个CUDA核心，每个核心都能够在任何时候处理这样的操作。

### 缓存

考虑以下情况: 我们有一个中等的CPU核心，有4个核心，如 :numref:`fig_skylake` 所示，运行在2GHz频率。此外，我们假设IPC（每个时钟的指令数）计数为1，并且这些单元启用了256位宽度的AVX2。让我们进一步假设需要从存储器检索用于AVX2操作的至少一个寄存器。这意味着中央处理器每个时钟周期消耗$4 \times 256 \text{ bit} = 128 \text{ bytes}$的数据。除非我们能够每秒向处理器传输$2 \times 10^9 \times 128 = 256 \times 10^9$字节，否则处理单元将会耗尽。不幸的是，这种芯片的存储器接口仅支持20-40 Gb/s的数据传输，即少了一个数量级。解决方法是尽可能避免从内存加载新数据，而是将其本地缓存在CPU上。这就是缓存派上用场的地方。通常使用以下名称或概念：

* **寄存器** 严格来说，不是缓存的一部分。它们帮助传达指令。这就是说，CPU寄存器是CPU可以以时钟速度访问而不受任何延迟惩罚的存储位置。CPU有几十个寄存器。有效地使用寄存器取决于编译器（或程序员）。例如，C语言有一个`register`关键字。
* **一级缓存**是应对高内存带宽要求的第一道防线。一级缓存很小（常见的大小可能是32-64KB），通常分为数据和指令缓存。当在一级缓存中找到数据时，访问速度非常快。如果在那里找不到它们，搜索将沿着缓存层次结构向下进行。
* **二级缓存**是下一站。根据架构设计和处理器大小的不同，它们可能是独占的。即它们可能只能由给定的核心访问，或者在多个核心之间共享。二级缓存比一级缓存更大（通常每个核心256-512KB），而速度更慢。此外，要访问L2中的内容，我们首先需要检查以确定数据不在L1中，这会增加少量额外的延迟。
* **L3缓存**在多个核之间共享，并且可以非常大。AMD的Epyc 3服务器CPU在多个芯片上拥有高达256MB的高速缓存。更常见的数字在4-8MB范围内。

预测下一步需要哪些存储元件是芯片设计中的关键优化参数之一。例如，建议以向前的方向遍历内存，因为大多数缓存算法将尝试*向前读取*而不是向后读取。同样，将内存访问模式保持在本地也是提高性能的一个好方法。

添加缓存是一把双刃剑。一方面，它们确保处理器核心不缺乏数据。同时，它们增加了芯片尺寸，消耗了原本可以用来提高处理能力的面积。此外，*缓存未命中*的代价可能会很昂贵。考虑最坏的情况，*错误共享*（false sharing），如 :numref:`fig_falsesharing` 所示。当处理器1上的线程请求数据时，内存位置缓存在处理器0上。为了获得它，处理器0需要停止它正在做的事情，将信息写回主内存，然后让处理器1从内存中读取它。在此操作期间，两个处理器都等待。与高效的单处理器实现相比，这种代码在多个处理器上运行的速度可能要慢得多。这就是为什么缓存大小（除了物理大小之外）有实际限制的另一个原因。

![错误共享（图片由英特尔提供）](../img/falsesharing.svg)
:label:`fig_falsesharing`

## GPU和其他加速卡

毫不夸张地说，如果没有GPU，深度学习就不会成功。基于同样的原因，有理由认为GPU制造商的财富由于深度学习而显著增加。这种硬件和算法的协同进化导致了这样一种情况：无论好坏，深度学习都是更可取的统计建模范式。因此，了解GPU和其他加速卡（如TPU :cite:`Jouppi.Young.Patil.ea.2017` ）的具体好处是值得的。

值得注意的是，在实践中经常会有这样一个区别：加速卡是为训练或推理而优化的。对于后者，我们只需要计算网络中的前向传播。反向传播不需要存储中间数据。此外，我们可能不需要非常精确的计算（FP16或INT8通常就足够了）。另一方面，在训练过程中，所有中间结果都需要存储来计算梯度。此外，累积梯度需要更高的精度，以避免数值下溢（或溢出）。这意味着FP16（或与FP32的混合精度）是最低要求。所有这些都需要更快、更大的内存（HBM2与GDDR6相比）和更高的处理能力。例如，NVIDIA的[Turing](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/) T4 GPU优化用于推理，而V100 GPU更适合用于训练。

回想一下如 :numref:`fig_neon128` 所示的矢量化。将向量处理单元添加到处理器核心可以显著提高吞吐量。例如，在 :numref:`fig_neon128` 的例子中，我们能够同时执行16个操作。首先，如果我们添加的运算不仅优化了向量之间的运算，而且优化了矩阵之间的运算，会怎么样？这个策略引入了张量核（tensor cores），这稍后将讨论。第二，如果我们增加更多的核心呢？简而言之，这两种策略总结了GPU中的设计决策。 :numref:`fig_turing_processing_block` 给出了基本处理块的概述。它包含16个整数和16个浮点单位。除此之外，两个张量核加速了与深度学习相关的附加操作的窄子集。每个流式多处理器由四个这样的块组成。

![NVIDIA Turing 处理块（图片由英伟达提供）](../img/turing-processing-block.png)
:width:`150px`
:label:`fig_turing_processing_block`

接下来，将12个流式多处理器分组为图形处理集群，这些集群构成了高端TU102处理器。充足的内存通道和二级缓存补充了设置。 :numref:`fig_turing` 有相关的细节。设计这种设备的原因之一是，可以根据需要添加或删除单个模块，以允许更紧凑的芯片和处理成品率问题（故障模块可能无法激活）。幸运的是，在CUDA和框架代码层之下，这类设备的编程对随意的深度学习研究人员隐藏得很好。特别是，如果有可用的资源，在GPU上可以同时执行多个程序。尽管如此，了解设备的局限性是值得的，以避免选择不适合设备内存的型号。

![NVIDIA Turing 架构（图片由英伟达提供）](../img/turing.png)
:width:`350px`
:label:`fig_turing`

最后值得一提的是*张量核*（tensor cores）。它们是最近增加更多优化电路的趋势的一个例子，这些电路对深度学习特别有效。例如，TPU添加了用于快速矩阵乘法的脉动阵列:cite:`Kung.1988`。在那里，设计是为了支持非常小数量（第一代TPU支持数量为1）的大型操作。张量核是另一个极端。它们针对$4 \times 4$和$16 \times 16$矩阵之间的小型运算进行了优化，具体取决于它们的数值精度。 :numref:`fig_tensorcore` 给出了优化的概述。

![NVIDIA Turing架构中的张量核心（图片由英伟达提供）](../img/tensorcore.jpg)
:width:`400px`
:label:`fig_tensorcore`

显然，在针对计算进行优化时，我们最终会做出某些妥协。其中之一是GPU不太擅长处理中断和稀疏数据。尽管有一些明显的例外，如[Gunrock](https://github.com/gunrock/gunrock) :cite:`Wang.Davidson.Pan.ea.2016`，但稀疏矩阵和向量的访问模式并不适合GPU擅长的高带宽突发读取操作。匹配这两个目标是一个积极研究的领域。例如[DGL](http://dgl.ai)，这是一个专为图深度学习而设计的库。

## 网络和总线

每当单个设备不足以进行优化时，我们就需要在其中来回传输数据以同步处理。这就是网络和总线派上用场的地方。我们有许多设计参数：带宽、成本、距离和灵活性。一方面，我们有一个很好的WiFi范围，是非常容易使用（毕竟没有线缆），便宜，但它提供的带宽和延迟相对一般。没有一个头脑正常的机器学习研究人员会用它来构建服务器集群。在接下来的内容中，我们将重点关注适合深度学习的互连。

* **PCIe**是一种专用总线，用于每个通道的高带宽点到点连接（在16通道插槽中的PCIe 4.0上高达32 GB/s）。延迟时间为个位数的微秒（5μs） PCIe链接非常宝贵。处理器的数量有限：AMD的EPYC 3有128个通道，Intel的Xeon每个芯片有48个通道；在桌面级CPU上，数字分别是20（Ryzen 9）和16（Core i9）。由于GPU通常有16个通道，这限制了可以以全带宽连接到CPU的GPU数量。毕竟，它们需要与其他高带宽外围设备（如存储和以太网）共享链路。与内存访问一样，由于减少了数据包开销，因此更适合大容量传输。
* **以太网**是连接计算机最常用的方式。虽然它比PCIe慢得多，但它的安装成本非常低，而且具有很强的弹性，而且覆盖的距离要长得多。低级服务器的典型带宽为1 GBit/s。高端设备（如云中的[C5实例](https://aws.amazon.com/ec2/instance-types/c5/)）提供10到100GBit/s的带宽。与所有以前的情况一样，数据传输有很大的开销。请注意，我们几乎从不直接使用原始以太网，而是使用在物理互连之上执行的协议(例如UDP或TCP/IP)。这进一步增加了开销。与PCIe类似，以太网旨在连接两个设备，例如计算机和交换机。
* **交换机**允许我们以一种方式连接多个设备，该连接方式下的任何一对设备都可以同时执行（通常是全带宽）点对点连接。例如，以太网交换机可能以高带宽连接40台服务器。请注意，交换机并不是传统计算机网络所独有的。甚至PCIe通道也可以是[可交换的](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)。例如，将大量GPU连接到主机处理器时会出现这种情况，[P2实例](https://aws.amazon.com/ec2/instance-types/p2/)就是这种情况。
* **NVLink**是PCIe的替代品，适用于非常高带宽的互连。它为每条链路提供高达300 Gbit/s的数据传输速率。服务器GPU（Volta V100）有六个链路。而消费级GPU（RTX 2080 Ti）只有一个链路，运行速度降低到100 Gbit/s。我们建议使用[NCCL](https://github.com/NVIDIA/nccl)来实现GPU之间的高速数据传输。

## 更多延迟

 :numref:`table_latency_numbers` 和 :numref:`table_latency_numbers_tesla` 中的小结来自[Eliot Eshelman](https://gist.github.com/eshelman)，他们将数字的更新版本保存到[GitHub gist](https://gist.github.com/eshelman/343a1c46cb3fba142c1afdcdeec17646)。

:常见延迟。

| Action | Time | Notes |
| :----------------------------------------- | -----: | :---------------------------------------------- |
| L1 cache reference/hit                     | 1.5 ns | 4 cycles                                        |
| Floating-point add/mult/FMA                | 1.5 ns | 4 cycles                                        |
| L2 cache reference/hit                     |   5 ns | 12 ~ 17 cycles                                  |
| Branch mispredict                          |   6 ns | 15 ~ 20 cycles                                  |
| L3 cache hit (unshared cache)              |  16 ns | 42 cycles                                       |
| L3 cache hit (shared in another core)      |  25 ns | 65 cycles                                       |
| Mutex lock/unlock                          |  25 ns |                                                 |
| L3 cache hit (modified in another core)    |  29 ns | 75 cycles                                       |
| L3 cache hit (on a remote CPU socket)      |  40 ns | 100 ~ 300 cycles (40 ~ 116 ns)                  |
| QPI hop to a another CPU (per hop)         |  40 ns |                                                 |
| 64MB memory ref. (local CPU)          |  46 ns | TinyMemBench on Broadwell E5-2690v4             |
| 64MB memory ref. (remote CPU)         |  70 ns | TinyMemBench on Broadwell E5-2690v4             |
| 256MB memory ref. (local CPU)         |  75 ns | TinyMemBench on Broadwell E5-2690v4             |
| Intel Optane random write                  |  94 ns | UCSD Non-Volatile Systems Lab                   |
| 256MB memory ref. (remote CPU)        | 120 ns | TinyMemBench on Broadwell E5-2690v4             |
| Intel Optane random read                   | 305 ns | UCSD Non-Volatile Systems Lab                   |
| Send 4KB over 100 Gbps HPC fabric          |   1 μs | MVAPICH2 over Intel Omni-Path                   |
| Compress 1KB with Google Snappy            |   3 μs |                                                 |
| Send 4KB over 10 Gbps ethernet             |  10 μs |                                                 |
| Write 4KB randomly to NVMe SSD             |  30 μs | DC P3608 NVMe SSD (QOS 99% is 500μs)            |
| Transfer 1MB to/from NVLink GPU            |  30 μs | ~33GB/s on NVIDIA 40GB NVLink                 |
| Transfer 1MB to/from PCI-E GPU             |  80 μs | ~12GB/s on PCIe 3.0 x16 link                  |
| Read 4KB randomly from NVMe SSD            | 120 μs | DC P3608 NVMe SSD (QOS 99%)                     |
| Read 1MB sequentially from NVMe SSD        | 208 μs | ~4.8GB/s DC P3608 NVMe SSD                    |
| Write 4KB randomly to SATA SSD             | 500 μs | DC S3510 SATA SSD (QOS 99.9%)                   |
| Read 4KB randomly from SATA SSD            | 500 μs | DC S3510 SATA SSD (QOS 99.9%)                   |
| Round trip within same datacenter          | 500 μs | One-way ping is ~250μs                          |
| Read 1MB sequentially from SATA SSD        |   2 ms | ~550MB/s DC S3510 SATA SSD                    |
| Read 1MB sequentially from disk            |   5 ms | ~200MB/s server HDD                           |
| Random Disk Access (seek+rotation)         |  10 ms |                                                 |
| Send packet CA->Netherlands->CA            | 150 ms |                                                 |
:label:`table_latency_numbers`

:NVIDIA Tesla GPU的延迟.

| Action | Time | Notes |
| :------------------------------ | -----: | :---------------------------------------- |
| GPU Shared Memory access        |  30 ns | 30~90 cycles (bank conflicts add latency) |
| GPU Global Memory access        | 200 ns | 200~800 cycles                            |
| Launch CUDA kernel on GPU       |  10 μs | Host CPU instructs GPU to start kernel    |
| Transfer 1MB to/from NVLink GPU |  30 μs | ~33GB/s on NVIDIA 40GB NVLink           |
| Transfer 1MB to/from PCI-E GPU  |  80 μs | ~12GB/s on PCI-Express x16 link         |
:label:`table_latency_numbers_tesla`

## 小结

* 设备有运行开销。因此，重要的是要争取少量的大数据传送，而不是许多小规模的数据传送。这适用于内存、固态驱动器、网络和GPU。
* 矢量化是性能的关键。确保您了解加速器的特定功能。例如，一些Intel Xeon CPU特别适用于INT8操作，NVIDIA Volta GPU擅长FP16矩阵操作，NVIDIA Turing擅长FP16、INT8和INT4操作。
* 小数据类型导致的数值溢出在训练过程中可能是个问题（在推理过程中的影响程度较小）。
* 64位CPU上的内存对齐应该按照64位边界进行。在GPU上，保持卷积大小对齐是一个好主意，例如与张量核对齐。
* 将算法与硬件相匹配（例如，内存占用和带宽）。将参数装入缓存时，可以实现很大的加速比(数量级)。
* 我们建议你在验证实验结果之前先在纸上勾勒出新算法的性能。数量级或更大数量级的差异是令人担忧的原因。
* 使用调试器调试性能瓶颈。
* 训练和推理硬件在价格和性能方面有不同的优点。

## 练习

1. 编写C代码来测试访问对齐的内存和未对齐的内存之间的速度是否有任何差异。提示：小心缓存效果。
1. 测试按顺序或给定步幅访问内存的速度差异。
1. 如何测量CPU上的缓存大小？
1. 如何在多个内存通道中布局数据以获得最大带宽？如果你有许多细线，你会怎么布置呢？
1. 一个企业级硬盘正在以10000转/分的速度旋转。在最坏的情况下，硬盘读取数据所需的最短时间是多少（你可以假设磁头几乎是瞬间移动的）？为什么2.5英寸硬盘在商用服务器上越来越流行（相对于3.5英寸和5.25英寸硬盘）？
1. 假设HDD制造商将存储密度从每平方英寸1 Tbit增加到每平方英寸5 Tbit。在一个2.5英寸的硬盘上，一个环能存储多少信息？内轨和外轨有区别吗？
1. 从8位数据类型到16位数据类型，硅片的数量大约增加了四倍，为什么？为什么NVIDIA会在其图灵GPU中添加INT4运算？
1.在内存中向前读比向后读快多少？该数字在不同的计算机和CPU供应商之间是否有所不同？为什么？编写C代码并进行实验。
1. 你能测量一下磁盘的缓存大小吗？典型的硬盘是多少？固态驱动器需要缓存吗？
1. 测量通过以太网发送消息时的数据包开销。查找UDP和TCP/IP连接之间的差异。
1. 直接内存访问允许CPU以外的设备直接向内存写入（和读取）。为什么这是个好主意？
1. 看看Turing T4 GPU的性能数字。为什么从FP16到INT8和INT4的性能只翻倍？
1. 从旧金山到阿姆斯特丹的往返旅行，一个网络包需要多长时间？提示：你可以假设距离为10000公里。

[Discussions](https://discuss.d2l.ai/t/2798)
